In [2]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

park = Customer('Park', 100)

In [3]:
class LineItem:
  def __init__(self, product, quantity, price):
    self.product = product
    self.quantity = quantity
    self.price = price

  def total(self):
    return self.price * self.quantity

In [10]:
class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)  # 수정된 부분
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [11]:
class Promotion(ABC):
  @abstractmethod
  def discount(self, order):
    """할인액을 구체적인 숫자로 변환"""
    pass

In [12]:
class FidelityPromo(Promotion):
  def discount(self, order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion):
  def discount(self, order):
    discount = 0
    for item in order.cart:
      if item.quantity >= 20:
        discount += item.total() * 0.1
    return discount

class LargeOrderPromo(Promotion):
  def discount(self, order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
      return order.total() * 0.07
    return 0

In [13]:
joe = Customer('John Doe', 0)
ann = Customer('Ann smith', 1000)
cart = [
  LineItem('banana', 4, .5),
  LineItem('apple', 10, 1.5),
  LineItem('watermellon', 5, 5.0)
]

Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [14]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [15]:
banana_cart = [LineItem('banana', 30, .5), LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [16]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [18]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

In [23]:
class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


# 프로모션 함수 정의
def fidelity_promo(order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0


def bulk_item_promo(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount


def large_order_promo(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

In [24]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [25]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [26]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [27]:
Order(ann, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [29]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
  return max(promo(order) for promo in promos)

In [30]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [32]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [33]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

In [34]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', '_i2', 'ABC', 'abstractmethod', 'namedtuple', 'Customer', 'park', '_i3', 'LineItem', '_i4', 'Order', '_i5', 'Promotion', '_i6', 'FidelityPromo', 'BulkItemPromo', 'LargeOrderPromo', '_i7', 'joe', 'ann', 'cart', '_7', '_i8', '_i9', '_9', '_i10', '_i11', '_i12', '_i13', '_13', '_i14', '_14', '_i15', 'banana_cart', '_15', '_i16', 'long_order', '_16', '_i17', '_i18', '_18', '_i19', 'fidelity_promo', 'bulk_item_promo', 'large_order_promo', '_i20', '_20', '_i21', '_i22', '_22', '_i23', '_i24', '_24', '_i25', '_25', '_i26', '_26', '_i27', '_27', '_i28', 'promos', '_i29', 'best_promo', '_i30', '_30', '_i31', '_31', '_i32', '_32', '_i33', '_33', '_i34'])

In [35]:
globals()['best_promo']

<function __main__.best_promo(order)>

In [37]:
promos = [globals()[name] for name in globals() if name.endswith('_promo') and name != 'best_promo']
promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]

In [38]:
def deco(func):
  def inner():
    print('running inner()')
  return inner

In [40]:
def target():
  print("runnign target()")

target()

runnign target()


In [41]:
target

<function __main__.target()>

In [42]:
registry = []

def register(func):
  print('running register(%s)' % func)
  registry.append(func)
  return func

def f1():
  print('running f1()')

def f2():
  print('running f2()')

def f3():
  print('running f3()')

def main():
  print('running main()')
  print('registry ->', registry)
  f1()
  f2()
  f3()

if __name__ == '__main__':
  main()

running main()
registry -> []
running f1()
running f2()
running f3()


In [43]:
def promotion(promo_func):
  promos.append(promo_func)
  return promo_func

def fidelity_promo(order):
  return order.total() * 0.05 if order.customer.fiedlity >= 1000 else 0

def bulk_item_promo(order):
  discount = 0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * 0.1
  return discount

def large_order_promo(order):
  distinct_items = {item.product for item in order.cart}
  if len(distinct_items) >= 10:
    return order.total() * 0.07
  return 0

def best_promo(order):
  return max(promo(order) for promo in promos)

In [44]:
def f1(a):
  print(a)
  print(b)

f1(3)

3


NameError: name 'b' is not defined

In [45]:
b=6
f1(3)

3
6


In [51]:
b = 6
def f2(a):
  print(a)
  print(b)
  b = 9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [50]:
b = 6
def f3(a):
  global b
  print(a)
  print(b)
  b = 9

f3(3)

3
6


In [52]:
b

6

In [53]:
f3(3)

3
6


In [54]:
b=30
f3(3)

3
30


In [55]:
class Average():
  def __init__(self):
    self.series = []

  def __call__(self, new_value):
    self.series.append(new_value)
    total = sum(self.series)
    return total / len(self.series)

In [56]:
avg = Average()
avg(10)

10.0

In [57]:
avg(11)

10.5

In [58]:
avg(12)

11.0

In [59]:
def make_averager():
  series = []

  def averager(new_value):
    series.append(new_value)
    total = sum(series)
    return total / len(series)

  return averager

In [60]:
avg = make_averager()
avg(10)

10.0

In [61]:
avg(11)

10.5

In [62]:
avg(12)

11.0

In [63]:
avg.__code__.co_varnames

('new_value', 'total')

In [64]:
avg.__code__.co_freevars

('series',)

In [65]:
avg.__closure__

(<cell at 0x7d43579dc7c0: list object at 0x7d43980f9f40>,)

In [66]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [67]:
def make_averager():
  count = 0
  total = 0
  def averager(new_value):
    count += 1
    total += new_value
    return total / count
  return averager

In [68]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [69]:
def make_averager():
  count = 0
  total = 0
  def averager(new_value):
    nonlocal count, total
    count += 1
    total += new_value
    return total / count
  return averager

In [70]:
avg = make_averager()
avg(10)

10.0

In [71]:
import time

def clock(func):
  def clocked(*args):
    t0 = time.perf_counter()
    result = func(*args)
    elapsed = time.perf_counter() - t0
    name = func.__name__
    arg_str = ', '.join(repr(arg) for arg in args)
    print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
    return result

In [72]:
def snooze(seconds):
  time.sleep(seconds)

def factorial(n):
  return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
  print('*'*40, 'Calling snooze(.123)')
  snooze(.123)
  print('*'*40, 'Calling factorial(6)')
  print('6! =', factorial(6))

**************************************** Calling snooze(.123)
**************************************** Calling factorial(6)
6! = 720


In [73]:
factorial.__name__

'factorial'

In [76]:
import time
import functorch

def clock(func):
  @functools.wraps(func)
  def clocked(*args, **kwargs):
    t0 = time.time()
    result = func(*args, **kwargs)
    elapsed = time.time() - t0
    arg_lst =[]
    name = func.__name__
    if args:
      arg_lst.append(', '.join(repr(arg) for arg in args))
    if kwargs:
      pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
      arg_lst.append(', '.join(pairs))
    arg_str = ', '.join(arg_lst)
    print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
    return result
  return clocked

In [77]:
def snooze(seconds):
  time.sleep(seconds)

def factorial(n):
  return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
  print('*'*40, 'Calling snooze(.123)')
  snooze(.123)
  print('*'*40, 'Calling factorial(6)')
  print('6! =', factorial(6))

**************************************** Calling snooze(.123)
**************************************** Calling factorial(6)
6! = 720
